In [1]:
import pandas as pd

In [2]:
train_url = "https://github.com/neurone0/explore_data/raw/main/melbourne-housing-snapshot/Data/train.csv"
test_url = "https://github.com/neurone0/explore_data/raw/main/melbourne-housing-snapshot/Data/test.csv"

In [5]:
#Read the data:
X_full = pd.read_csv(train_url,index_col= "Id")
X_test_full = pd.read_csv(test_url,index_col="Id")

In [10]:
# Remove rows with missing target:
X_full.dropna(axis=0,subset= ['SalePrice'],inplace= True)
#separate target from predictors
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis= 1, inplace= True)

In [13]:
#we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

In [14]:
# Break off validation set from training data
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size=0.8,test_size= 0.2, random_state= 0)

In [15]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [20]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)
# Number of missing values in each column of training data
missing_value_count = (X_train.isnull().sum())
print(missing_value_count[missing_value_count>0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [21]:
# function reports the mean absolute error (MAE) from a random forest model.
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [28]:
#Function comparing different approach:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor()
    model.fit(X_train,y_train)
    pred = model.predict(X_valid)
    return mean_absolute_error(y_valid,pred)

In [29]:
#Drop column with missing values
#get names of columns with missing values
missingcols = [col for col in X_train.columns if X_train[col].isnull().any()]
missingcols

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [30]:
#drop columns in training and validation data
reduced_X_train = X_train.drop(missingcols, axis= 1)
reduced_X_valid = X_valid.drop(missingcols, axis= 1)

In [31]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17779.425662100457


In [32]:
#Imputation

In [33]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.fit_transform(X_valid))
print(imputed_X_train.head())
print(imputed_X_valid.head())

     0           1        2    3    4       5       6      7       8    9   \
0  20.0   90.000000  11694.0  9.0  5.0  2007.0  2007.0  452.0    48.0  0.0   
1  20.0   60.000000   6600.0  5.0  5.0  1962.0  1962.0    0.0     0.0  0.0   
2  30.0   80.000000  13360.0  5.0  7.0  1921.0  2006.0    0.0   713.0  0.0   
3  20.0   69.614017  13265.0  8.0  5.0  2002.0  2002.0  148.0  1218.0  0.0   
4  20.0  118.000000  13704.0  7.0  5.0  2001.0  2002.0  150.0     0.0  0.0   

   ...     26     27     28    29   30     31   32   33   34      35  
0  ...  774.0    0.0  108.0   0.0  0.0  260.0  0.0  0.0  7.0  2007.0  
1  ...  308.0    0.0    0.0   0.0  0.0    0.0  0.0  0.0  8.0  2009.0  
2  ...  432.0    0.0    0.0  44.0  0.0    0.0  0.0  0.0  8.0  2009.0  
3  ...  857.0  150.0   59.0   0.0  0.0    0.0  0.0  0.0  7.0  2008.0  
4  ...  843.0  468.0   81.0   0.0  0.0    0.0  0.0  0.0  1.0  2006.0  

[5 rows x 36 columns]
      0         1        2    3    4       5       6           7       8   \
0   2

In [34]:
#imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
print(imputed_X_train.head())
print(imputed_X_valid.head())



   MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0        20.0    90.000000  11694.0          9.0          5.0     2007.0   
1        20.0    60.000000   6600.0          5.0          5.0     1962.0   
2        30.0    80.000000  13360.0          5.0          7.0     1921.0   
3        20.0    69.614017  13265.0          8.0          5.0     2002.0   
4        20.0   118.000000  13704.0          7.0          5.0     2001.0   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  ...  GarageArea  \
0        2007.0       452.0        48.0         0.0  ...       774.0   
1        1962.0         0.0         0.0         0.0  ...       308.0   
2        2006.0         0.0       713.0         0.0  ...       432.0   
3        2002.0       148.0      1218.0         0.0  ...       857.0   
4        2002.0       150.0         0.0         0.0  ...       843.0   

   WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  \
0         0.0        108.0     

In [35]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18185.359543378996


In [36]:
# Preprocessed training and validation features
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

In [37]:
# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [38]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379


In [40]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Get test predictions
preds_test = model.predict(final_X_test)

C:\Users\subra\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [41]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
# output.to_csv('submission.csv', index=False)